In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import HTTP
import JSON

In [ ]:
ALL_MEDIUMS = ["manga", "anime"];

In [ ]:
URLS = Dict(
    "fetch_media_lists" => "http://localhost:3000",
    "compress_media_lists" => "http://localhost:3001",
    "nondirectional" => "http://localhost:3002",
    "transformer_jl" => "http://localhost:3003",
    "transformer_py" => "http://localhost:3004",
);

In [ ]:
run(app, path) = HTTP.get(URLS[app] * path)
run(app, path, json) =
    HTTP.post(URLS[app] * path, [("Content-Type", "application/json")], json);

In [ ]:
function wake()
    Threads.@threads for x in collect(keys(URLS))
        run(x, "/wake")
    end
end;

In [ ]:
function get_media_list(username, source)
    responses = Dict{Any,Any}(x => nothing for x in ALL_MEDIUMS)
    Threads.@threads for medium in ALL_MEDIUMS
        r1 = run(
            "fetch_media_lists",
            "/query?username=$username&source=$source&medium=$medium",
        )
        r2 = run(
            "compress_media_lists",
            "/query?username=$username&source=$source&medium=$medium",
            String(r1.body),
        )
        responses[medium] = JSON.parse(String(r2.body))
    end
    JSON.json(responses)
end;

In [ ]:
function transformer(data)
    r = run("transformer_jl", "/process", data)
    responses = Dict{Any,Any}(x => nothing for x in ALL_MEDIUMS)
    Threads.@threads for m in ALL_MEDIUMS
        responses[m] = run("transformer_py", "/query?medium=$m", String(copy(r.body)))
    end
    embeddings = merge([JSON.parse(String(copy(x.body))) for x in values(responses)]...)
    run(
        "transformer_jl",
        "/compute",
        JSON.json(Dict("payload" => JSON.parse(data), "embeddings" => embeddings)),
    )
end;

In [ ]:
function nondirectional(data)
    run("nondirectional", "/query", data)
end;

In [ ]:
function get_recs(username, source)
    @time data = get_media_list(username, source)
    @time t = transformer(data)
    @time n = nondirectional(data)
    t, n
end;

In [ ]:
data = get_recs("Fro116", "mal")